In [1]:
# EuRoC data handler
import pandas as pd
import numpy as np
import tensorflow as tf
# from keras import backend as K
import time
import sys

# class euroc_data():
# 	def __init__(self,groundtruth_path):
# 		gt_frame = pd.read_csv(groundtruth_path,'r')
GROUNDTRUTH_PATH = '/home/cdeng/EuRoC/mav0/\
state_groundtruth_estimate0/data.csv'
IMU_PATH = '/home/cdeng/EuRoC/mav0/imu0/data.csv'

In [2]:
gt_dataframe = pd.read_csv(GROUNDTRUTH_PATH, index_col=0)
imu_dataframe = pd.read_csv(IMU_PATH, index_col=0)
# print(gt_dataframe.head(3))
# print('-----------------')
# print(imu_dataframe.head(3))

# Find the start and end time stamp of the ground truth, 
# and extract relevant imu data frame, gt and imu should be same length  
groudtruth_data = gt_dataframe.iloc[:-1,0:7]
start_time, end_time = groudtruth_data.index[0], groudtruth_data.index[-1]

# Use groundtruth_data.iloc[0,:] as reference cordinate frame
groudtruth_data_rela_pos = groudtruth_data.iloc[:,:]-groudtruth_data.iloc[0,:]
print (start_time, end_time)
print(groudtruth_data_rela_pos.shape)
groudtruth_data.head(5)

1403636580838555648 1403636762738555392
(36381, 7)


,p_RS_R_x [m],p_RS_R_y [m],p_RS_R_z [m],q_RS_w [],q_RS_x [],q_RS_y [],q_RS_z []
#timestamp,,,,,,,
1403636580838555648,4.688319,-1.786938,0.783338,0.534108,-0.153029,-0.827383,-0.082152
1403636580843555328,4.688177,-1.786770,0.787350,0.534640,-0.152990,-0.826976,-0.082863
1403636580848555520,4.688028,-1.786598,0.791382,0.535178,-0.152945,-0.826562,-0.083605
1403636580853555456,4.687878,-1.786421,0.795429,0.535715,-0.152884,-0.826146,-0.084391
1403636580858555648,4.687727,-1.786240,0.799484,0.536244,-0.152821,-0.825731,-0.085213


In [3]:
# ***** Handling imu data ***************
# Set threshold to 2 000 000 ns, because the time interval between imu data is 5 000 000 ns
threshold = 1000000
[imu_start_idx, imu_end_idx] = [idx for (idx, timestamp) in enumerate(imu_dataframe.index)  \
    if (abs(timestamp-start_time)<threshold or abs(timestamp-end_time)<threshold)]
print (imu_start_idx, imu_end_idx)
# ---------imu data at t-1 is used to predicate gt at t
# Groundtruth is for image 1-n, [first_image_middle_exp_time, last_image_middle_exp_time], that't why the number of 
# datapoints end with 1.
imu_data = imu_dataframe[imu_start_idx:imu_end_idx].copy()

if (groudtruth_data_rela_pos.shape[0]-1 != imu_data.shape[0]):
    print("'Error: groundtruth and imu data don't have the same lenght'")
print (imu_data.shape)
print (groudtruth_data_rela_pos.shape)

groudtruth_data_rela_pos.iloc[:32,:]
# imu_data.iloc[1:3]
# imu_dataframe[0:6]

216 36596
(36380, 6)
(36381, 7)


,p_RS_R_x [m],p_RS_R_y [m],p_RS_R_z [m],q_RS_w [],q_RS_x [],q_RS_y [],q_RS_z []
#timestamp,,,,,,,
1403636580838555648,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1403636580843555328,-0.000142,0.000168,0.004012,0.000532,0.000039,0.000407,-0.000711
1403636580848555520,-0.000291,0.000340,0.008044,0.001070,0.000084,0.000821,-0.001453
1403636580853555456,-0.000441,0.000517,0.012091,0.001607,0.000145,0.001237,-0.002239
1403636580858555648,-0.000592,0.000698,0.016146,0.002136,0.000208,0.001652,-0.003061
1403636580863555328,-0.000740,0.000879,0.020202,0.002660,0.000261,0.002069,-0.003897
1403636580868555520,-0.000884,0.001057,0.024256,0.003181,0.000304,0.002487,-0.004738
1403636580873555456,-0.001024,0.001229,0.028304,0.003696,0.000349,0.002902,-0.005573
1403636580878555648,-0.001161,0.001394,0.032344,0.004209,0.000402,0.003316,-0.006401


In [4]:
imu_data.head(4)

,w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
#timestamp [ns],,,,,,
1403636580838555392,-0.206647,0.325329,-0.040492,10.035472,-0.179789,-4.135137
1403636580843555584,-0.215025,0.333009,-0.046775,9.749445,-0.024517,-4.077932
1403636580848555520,-0.224798,0.336499,-0.057945,9.496106,0.024517,-4.077932
1403636580853555456,-0.237365,0.338594,-0.074700,9.210079,0.032689,-4.061588


In [5]:
# After studying the dataset, ignore the last data from ground truth. Because the second last element 
# has the timestamp that happens to be the middle point of image 3661 and 3662 (opened in Office, column index)
datasize = imu_data.shape[0] 

# IMU 200Hz,   1 min = 12000 samples
series_length = 400 # This is 2 seconds
total_num_sequence = datasize//series_length
total_used_size = total_num_sequence*series_length

X = np.array(imu_data[:total_used_size])
y = np.empty(groudtruth_data_rela_pos[:total_used_size].shape)

# The first row of groudtruth_data_rela_pos is already relative displacement to last position
y[0,:] = groudtruth_data_rela_pos.iloc[0,:]
for i in range(1, y.shape[0] ):
    y[i,:3] = groudtruth_data_rela_pos.iloc[i,:3] - groudtruth_data_rela_pos.iloc[i-1,:3]
    y[i,3:] = groudtruth_data_rela_pos.iloc[i,3:]

# Handle extreme small number
y[y<1e-6] = 1e-6
    
input_dim = X.shape[1]
output_dim = y.shape[1] 

X = X.reshape(total_num_sequence, series_length, input_dim)
y = y.reshape(total_num_sequence, series_length, output_dim)


# print ('Training input data shape:',train_X.shape)
# print ('Testing output data shape:',train_Y.shape)

# Use relative position here
# X = X[:, :, :3]
# y = y[:, :, :3]
print ('All input data shape:', X.shape)
print ('All output data shape:', y.shape)

train_val_split_ratio = 0.8
train_num_sequence = round( total_num_sequence*train_val_split_ratio)

train_X = np.copy(X[:train_num_sequence,...])
train_Y = np.copy(y[:train_num_sequence,...])

# train_val_X = train_X.reshape(1,-1,input_dim)
# train_val_Y = train_Y.reshape(1,-1,output_dim)

test_X = np.copy(X[train_num_sequence:,...])
test_Y = np.copy(y[train_num_sequence:,...])

new_train_X = np.concatenate((train_X, train_Y), axis=2)
new_train_Y = train_Y[:,1:,:]

All input data shape: (90, 400, 6)
All output data shape: (90, 400, 7)


In [6]:
# for indexi,i in enumerate(y):
#     for indexj,j in enumerate(i):
#         if (abs(j)<0.000001).any():
#             print(j, indexi, indexj)
#             print ('------')

In [7]:
# print(y[0,29,:])
# test_X.shape

In [8]:
from random import randint
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.rnn import BasicLSTMCell

log_dir = '../tmp/logs/sensor_fusion_LSTM_rel_pos'
training_steps = 1000
drift_thredshold = 0.5
batch_size = 4

class RollingData(object):
    ''' X should be of shape (num_of_batches, sequence_length, input_dim)
        Shuffle the data!!! Small training batches!!!
    '''
    def __init__(self, X, Y,):
        self.pointer = 0
        self.X = X
        self.Y = Y
        self._num_of_samples = X.shape[0]
#         self.train_len = X.shape[1]/2
    
    @property
    def num_of_samples(self):
        return self._num_of_samples

    def next_batch(self):
        # Generator random number in the range self._num_of_samples
        index = np.random.choice(self._num_of_samples, 4, replace=False)
        batch_X = self.X[index,:,:]
        batch_Y = self.Y[index,:,:]        
#         try:
#             train_X = self.X[self.pointer, :self.train_len, :]
#             train_Y = self.Y[self.pointer, :self.train_len, :]
#             test_X  = self.X[self.pointer, self.train_len:, :]
#             test_Y  = self.Y[self.pointer, self.train_len:, :]
#             self.pointer += 1
#         except IndexError:
#             print ("Index out of range. Index range:{} Query index{}".format(self.num_of_batches, self.pointer))
#             sys.exit(1)
        return (batch_X, batch_Y)
    
#     def reset_pointer(self):
#         self.pointer = 0

class IMU_LSTM_Model(object):
    def __init__(self, lstm_cell=128, batch_size=1):

        self.lstm    = BasicLSTMCell(128)    
#         self.hidden_state = tf.zeros([batch_size, 64], dtype=tf.float32)
#         self.output_state = tf.zeros([batch_size, 64], dtype=tf.float32)
        self.state = (tf.zeros([batch_size, 128], dtype=tf.float32),tf.zeros([batch_size, 128], dtype=tf.float32))
        
#         self.batch_size = batch_size
        print('hello, good initialize')
#         self.warm_up_loss = 0
#         self.predict_loss = 0
#         self.loss = self.warm_up_loss + 9*self.predict_loss        
        
    def forward(self, X, mode=0):
#         with tf.variable_scope('fc1_before', reuse=True):
#             # Activation function here is tricky
# *** Change X 
        if mode ==0:
            self.X = X
        if mode ==1:
            self.X = tf.concat([X[:,:6],self.output], axis=1 )
        output1_before  = fully_connected(inputs=X, num_outputs=64, activation_fn= None)
#         with tf.variable_scope('fc2_before', reuse=True):
        output2_before  = fully_connected(inputs=output1_before, num_outputs=128, activation_fn= None)
#         with tf.variable_scope('lstm64',reuse=True):
        output_lstm, self.state = self.lstm(output2_before, self.state)
#         with tf.variable_scope('fc2_after',reuse=True):
        output_fc2 = fully_connected(inputs=output_lstm, num_outputs=128, activation_fn= None)
#         with tf.variable_scope('fc3_after',reuse=True):
        output_fc3 = fully_connected(inputs=output_fc2, num_outputs=64, activation_fn= None)
#         with tf.variable_scope('fc4_after', reuse=True):
        self.output = fully_connected(inputs=output_fc3, num_outputs=7, activation_fn= None)    #    
        return self.output
                
    # This method should be used in warm-up phase and between batches.
    def reset_state(self):
#         assert mode in (0,1), "mode should be either 0 or 1: {}".format(mode) 
        # Use mode 0 @ batch swtiching

        self.state = (tf.zeros([batch_size, 128], dtype=tf.float32),tf.zeros([batch_size, 128], dtype=tf.float32))
#             self.hidden_state = tf.zeros([batch_size, 64])
#             self.output_state = tf.zeros([batch_size, 64])
#             print(self.output_state)
        return self.state





In [12]:
from tensorflow.contrib.layers.python.layers import initializers
from tensorflow.python.ops import init_ops

In [ ]:
def inference_one_step(x,previous_hidden_memory_tuple):
    with tf.name_scope('fc1'):
        with tf.variable_scope('fc1_var', reuse=True, dtype=float32,):
            weights = tf.get_variable('weights', shape=[13,64],initializer=initializers.xavier_initializer())
            bias = tf.get_variable('bias', shape=[64],initializer=init_ops.zeros_initializer())
        fc1 = tf.matmul(x,weights)+bias
    with tf.name_scope('fc2'):
        with tf.variable_scope('fc2_var', reuse=True, dtype=float32,):
            weights = tf.get_variable('weights', shape=[64,128], initializer=initializers.xavier_initializer())
            bias = tf.get_variable('bias',  shape=[128],initializer=init_ops.zeros_initializer())
        fc2 = tf.matmul(x,weights)+bias
    with tf.name_scope('lstm128'):
        with tf.variable_scope('lstm128_var', reuse=True, dtype=float32):
            
                
# To be done!




def customized_loss(prediction, groundtruth, orientation_weight=0.10 ):
    # Orientation_weight should be decimal between 0 and 1, default=0.01
    pos_loss = tf.losses.mean_squared_error(prediction[:,:,:3],groundtruth[:,:,:3])
    # Step 1, normalize angle 
    normalized_orientation = tf.divide(prediction, tf.norm(prediction[:,:,3:], axis=2, keep_dims=True))
    ori_loss = tf.losses.mean_squared_error(normalized_orientation[:,:,3:], groundtruth[:,:,3:])
    # Step 2, get loss
    loss = 100*(pos_loss + orientation_weight*ori_loss)
    return loss, pos_loss, ori_loss


In [15]:
with tf.name_scope('aha'):
    with tf.variable_scope('var_scope_1', reuse=True):
        var1 = tf.get_variable(name='var1', shape=[1], dtype=tf.float32)
        var2 = tf.get_variable(name='var1', shape=[1], dtype=tf.float32)
    a = tf.add(var1, var2)
# with tf.name_scope('fuck'):
#     with tf.variable_scope('var_scope_1', reuse=True):
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var1.name, sess.run(var1))
    print(var2.name, sess.run(var2))
    print(a.name, sess.run(a))

    

var_scope_1/var1:0 [ 1.5455488]
var_scope_1/var1:0 [ 1.5455488]
aha_1/Add:0 [ 3.09109759]


In [9]:
tf.reset_default_graph()
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, shape=(batch_size, series_length, 13), name='x-input')
    Y = tf.placeholder(tf.float32, shape=(batch_size, series_length-1, 7), name='y-input')
#     mode = tf.placeholder(tf.int32, name='mode')
    
imu_lstm_model = IMU_LSTM_Model(batch_size=batch_size)
print(imu_lstm_model.lstm.state_size)


train_warm_up_pred = []
train_test_pred = []

# Because the last input, we don't have groudtruth, so we can not say its pred is right or wrong
for i in range(series_length-1):

    if i<series_length//2:
#         imu_lstm_model.set_state(mode=1, train_Y=Y[:,i,:])
#         train_test_pred.append(imu_lstm_model.forward(X[:,i,:]))  
#         _ = imu_lstm_model.set_state(mode=1, gt_train_Y = Y[:,series_length,:])
        train_warm_up_pred.append(imu_lstm_model.forward(X[:,i,:],  mode=0)) 
    else:
#         train_test_pred[:,i,:] = imu_lstm_model.forward(X[:,i,:])
        train_test_pred.append(imu_lstm_model.forward(X[:,i,:], mode=1)) 
#     if train_num_sequence == series_length-2:
#         zero_state = imu_lstm_model.reset_state()


print (len(train_warm_up_pred))
print (train_warm_up_pred[0].get_shape())

train_warm_up_pred = tf.reshape(train_warm_up_pred,[series_length//2,batch_size,output_dim])
train_warm_up_pred = tf.transpose(train_warm_up_pred, perm=[1,0,2])
train_test_pred = tf.reshape(train_test_pred,[series_length//2-1,batch_size,output_dim])
train_test_pred = tf.transpose(train_test_pred, perm=[1,0,2])

print (train_warm_up_pred.get_shape())


loss_warm_up, pos_loss_train, ori_loss_train = customized_loss(train_warm_up_pred,Y[:,:series_length//2,:])
loss_prediction, pos_loss, ori_loss = customized_loss(train_test_pred, Y[:,series_length//2:,:])
total_loss = 0.2*loss_warm_up + 0.8*loss_prediction

train_step = tf.train.AdamOptimizer().minimize(loss_prediction)

hello, good initialize
LSTMStateTuple(c=128, h=128)
200
(4, 7)
(4, 200, 7)


In [10]:
#-------***************--------------#

tf.summary.scalar('total_loss',total_loss)
tf.summary.scalar('loss_warm_up', loss_warm_up)
tf.summary.scalar('pos_loss_train', pos_loss_train)
tf.summary.scalar('ori_loss_train', ori_loss_train)
tf.summary.scalar('pred_loss',loss_prediction)
tf.summary.scalar('pos_loss',pos_loss)
tf.summary.scalar('ori_loss', ori_loss)
merged = tf.summary.merge_all()

saver = tf.train.Saver()


sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
train_rolling = RollingData(new_train_X, new_train_Y)
train_writer = tf.summary.FileWriter(log_dir+'/train', sess.graph, flush_secs=2, max_queue=2)


for e in range(3000):
    print ('Epoch {}/{} '.format(e, training_steps), end='\r')
    train_X_batch, train_Y_batch = train_rolling.next_batch()
    
    _,pred_loss, _= sess.run([train_step,loss_prediction, total_loss], feed_dict={X:train_X_batch, Y:train_Y_batch})
    zero_state = sess.run(imu_lstm_model.reset_state())
    print ("Prediction loss at step {} is: {}".format(e, pred_loss))
#     print ('Output state at the end of step {} is {}'.format(e, zero_state))

    summary,  = sess.run([merged], feed_dict={X:train_X_batch, Y:train_Y_batch})
    train_writer.add_summary(summary,e) 
    if e%300 ==0:
        save_path = saver.save(sess,'../tmp/logs/sensor_fusion_LSTM_rel_pos/model_%d.ckpt'%(e))
#     print('Hello test positioni correct ratio:{} \t and loss:{}'.format(test_correct_ratio, test_loss))
    
sess.close()

Prediction loss at step 0 is: 11.657685279846191
Prediction loss at step 1 is: 7.765103816986084
Prediction loss at step 2 is: 8.129000663757324
Prediction loss at step 3 is: 6.060035705566406
Prediction loss at step 4 is: 4.514664649963379
Prediction loss at step 5 is: 3.8734469413757324
Prediction loss at step 6 is: 3.7855207920074463
Prediction loss at step 7 is: 3.6128692626953125
Prediction loss at step 8 is: 3.4527950286865234
Prediction loss at step 9 is: 3.5951004028320312
Prediction loss at step 10 is: 3.4765264987945557
Prediction loss at step 11 is: 2.778843402862549
Prediction loss at step 12 is: 2.7393362522125244
Prediction loss at step 13 is: 2.735983371734619
Prediction loss at step 14 is: 2.678969383239746
Prediction loss at step 15 is: 2.7016830444335938
Prediction loss at step 16 is: 2.7466416358947754
Prediction loss at step 17 is: 2.5328562259674072
Prediction loss at step 18 is: 2.2545382976531982
Prediction loss at step 19 is: 2.152743101119995
Prediction loss at

Prediction loss at step 164 is: 2.137591600418091
Prediction loss at step 165 is: 2.0832488536834717
Prediction loss at step 166 is: 2.0965681076049805
Prediction loss at step 167 is: 2.3630459308624268
Prediction loss at step 168 is: 1.9968618154525757
Prediction loss at step 169 is: 1.7756319046020508
Prediction loss at step 170 is: 2.1222078800201416
Prediction loss at step 171 is: 2.0475499629974365
Prediction loss at step 172 is: 1.7491616010665894
Prediction loss at step 173 is: 1.7413345575332642
Prediction loss at step 174 is: 1.988588809967041
Prediction loss at step 175 is: 2.244856834411621
Prediction loss at step 176 is: 2.0974695682525635
Prediction loss at step 177 is: 2.0104825496673584
Prediction loss at step 178 is: 1.9529356956481934
Prediction loss at step 179 is: 1.713773250579834
Prediction loss at step 180 is: 2.2344374656677246
Prediction loss at step 181 is: 1.3044564723968506
Prediction loss at step 182 is: 1.7870724201202393
Prediction loss at step 183 is: 1.8

Prediction loss at step 326 is: 1.5762590169906616
Prediction loss at step 327 is: 1.5678297281265259
Prediction loss at step 328 is: 1.855713129043579
Prediction loss at step 329 is: 1.742191195487976
Prediction loss at step 330 is: 1.7737157344818115
Prediction loss at step 331 is: 1.266339898109436
Prediction loss at step 332 is: 1.9280602931976318
Prediction loss at step 333 is: 1.4582470655441284
Prediction loss at step 334 is: 1.6939131021499634
Prediction loss at step 335 is: 2.0568859577178955
Prediction loss at step 336 is: 1.9742438793182373
Prediction loss at step 337 is: 1.5466301441192627
Prediction loss at step 338 is: 1.4629850387573242
Prediction loss at step 339 is: 1.8289432525634766
Prediction loss at step 340 is: 1.8328497409820557
Prediction loss at step 341 is: 1.8019044399261475
Prediction loss at step 342 is: 2.0353429317474365
Prediction loss at step 343 is: 1.4699201583862305
Prediction loss at step 344 is: 1.3642178773880005
Prediction loss at step 345 is: 1.

Prediction loss at step 488 is: 1.6307188272476196
Prediction loss at step 489 is: 1.563658595085144
Prediction loss at step 490 is: 1.8850194215774536
Prediction loss at step 491 is: 1.8960856199264526
Prediction loss at step 492 is: 1.657232403755188
Prediction loss at step 493 is: 1.570949673652649
Prediction loss at step 494 is: 1.314106822013855
Prediction loss at step 495 is: 1.9751026630401611
Prediction loss at step 496 is: 2.0263190269470215
Prediction loss at step 497 is: 1.4716036319732666
Prediction loss at step 498 is: 1.9038667678833008
Prediction loss at step 499 is: 2.1471915245056152
Prediction loss at step 500 is: 1.9924521446228027
Prediction loss at step 501 is: 2.110089063644409
Prediction loss at step 502 is: 1.576744556427002
Prediction loss at step 503 is: 1.5584588050842285
Prediction loss at step 504 is: 2.056501626968384
Prediction loss at step 505 is: 1.5716357231140137
Prediction loss at step 506 is: 1.5858089923858643
Prediction loss at step 507 is: 1.2942

Prediction loss at step 650 is: 1.5363517999649048
Prediction loss at step 651 is: 1.6937295198440552
Prediction loss at step 652 is: 2.105013132095337
Prediction loss at step 653 is: 1.2515219449996948
Prediction loss at step 654 is: 0.9678918719291687
Prediction loss at step 655 is: 2.3330321311950684
Prediction loss at step 656 is: 1.9253572225570679
Prediction loss at step 657 is: 1.9951436519622803
Prediction loss at step 658 is: 2.1146352291107178
Prediction loss at step 659 is: 1.9297844171524048
Prediction loss at step 660 is: 1.7988739013671875
Prediction loss at step 661 is: 1.4922915697097778
Prediction loss at step 662 is: 2.120124340057373
Prediction loss at step 663 is: 1.7841557264328003
Prediction loss at step 664 is: 1.9775586128234863
Prediction loss at step 665 is: 1.6030043363571167
Prediction loss at step 666 is: 1.9930343627929688
Prediction loss at step 667 is: 1.9738521575927734
Prediction loss at step 668 is: 1.6519883871078491
Prediction loss at step 669 is: 1

...

Prediction loss at step 2245 is: 2.185793399810791
Prediction loss at step 2246 is: 1.977425217628479
Prediction loss at step 2247 is: 2.238436222076416
Prediction loss at step 2248 is: 2.002563714981079
Prediction loss at step 2249 is: 2.020176410675049
Prediction loss at step 2250 is: 1.2073156833648682
Prediction loss at step 2251 is: 1.8080909252166748
Prediction loss at step 2252 is: 1.990980863571167
Prediction loss at step 2253 is: 1.5775442123413086
Prediction loss at step 2254 is: 1.3917609453201294
Prediction loss at step 2255 is: 2.0530202388763428
Prediction loss at step 2256 is: 1.6622440814971924
Prediction loss at step 2257 is: 1.6942273378372192
Prediction loss at step 2258 is: 1.102113127708435
Prediction loss at step 2259 is: 1.3224120140075684
Prediction loss at step 2260 is: 2.126495838165283
Prediction loss at step 2261 is: 1.4048254489898682
Prediction loss at step 2262 is: 1.5447449684143066
Prediction loss at step 2263 is: 1.953201174736023
Prediction loss at st

Prediction loss at step 2404 is: 1.3529436588287354
Prediction loss at step 2405 is: 1.5981560945510864
Prediction loss at step 2406 is: 1.8156607151031494
Prediction loss at step 2407 is: 1.7669873237609863
Prediction loss at step 2408 is: 1.6204509735107422
Prediction loss at step 2409 is: 1.6958147287368774
Prediction loss at step 2410 is: 2.1804866790771484
Prediction loss at step 2411 is: 2.142524242401123
Prediction loss at step 2412 is: 1.6533746719360352
Prediction loss at step 2413 is: 1.9259039163589478
Prediction loss at step 2414 is: 1.5677671432495117
Prediction loss at step 2415 is: 2.0292975902557373
Prediction loss at step 2416 is: 1.4537068605422974
Prediction loss at step 2417 is: 1.8155643939971924
Prediction loss at step 2418 is: 1.5494240522384644
Prediction loss at step 2419 is: 2.0387091636657715
Prediction loss at step 2420 is: 1.9746713638305664
Prediction loss at step 2421 is: 1.3433464765548706
Prediction loss at step 2422 is: 1.7114143371582031
Prediction lo

Prediction loss at step 2563 is: 2.0575194358825684
Prediction loss at step 2564 is: 2.119204521179199
Prediction loss at step 2565 is: 1.6711689233779907
Prediction loss at step 2566 is: 1.9644865989685059
Prediction loss at step 2567 is: 2.1193270683288574
Prediction loss at step 2568 is: 1.4794440269470215
Prediction loss at step 2569 is: 2.0108535289764404
Prediction loss at step 2570 is: 2.061753034591675
Prediction loss at step 2571 is: 2.269681930541992
Prediction loss at step 2572 is: 2.0076398849487305
Prediction loss at step 2573 is: 1.9418730735778809
Prediction loss at step 2574 is: 1.382230281829834
Prediction loss at step 2575 is: 1.8981592655181885
Prediction loss at step 2576 is: 1.6699678897857666
Prediction loss at step 2577 is: 1.8630212545394897
Prediction loss at step 2578 is: 2.2584784030914307
Prediction loss at step 2579 is: 2.122802972793579
Prediction loss at step 2580 is: 2.2463531494140625
Prediction loss at step 2581 is: 2.136063575744629
Prediction loss at

Prediction loss at step 2722 is: 1.747593879699707
Prediction loss at step 2723 is: 1.9673707485198975
Prediction loss at step 2724 is: 1.316205382347107
Prediction loss at step 2725 is: 1.9230583906173706
Prediction loss at step 2726 is: 2.0114641189575195
Prediction loss at step 2727 is: 1.270478367805481
Prediction loss at step 2728 is: 1.5452063083648682
Prediction loss at step 2729 is: 1.6862335205078125
Prediction loss at step 2730 is: 1.830629825592041
Prediction loss at step 2731 is: 1.6469141244888306
Prediction loss at step 2732 is: 2.22019362449646
Prediction loss at step 2733 is: 1.8035190105438232
Prediction loss at step 2734 is: 1.9766453504562378
Prediction loss at step 2735 is: 2.054387092590332
Prediction loss at step 2736 is: 1.772512435913086
Prediction loss at step 2737 is: 1.9085605144500732
Prediction loss at step 2738 is: 1.8009718656539917
Prediction loss at step 2739 is: 1.982130527496338
Prediction loss at step 2740 is: 2.0239033699035645
Prediction loss at st

Prediction loss at step 2881 is: 2.084947347640991
Prediction loss at step 2882 is: 2.1637959480285645
Prediction loss at step 2883 is: 1.6575653553009033
Prediction loss at step 2884 is: 2.1195454597473145
Prediction loss at step 2885 is: 1.857672095298767
Prediction loss at step 2886 is: 2.1025712490081787
Prediction loss at step 2887 is: 2.023937702178955
Prediction loss at step 2888 is: 1.9078893661499023
Prediction loss at step 2889 is: 1.7667744159698486
Prediction loss at step 2890 is: 1.5940639972686768
Prediction loss at step 2891 is: 2.1114444732666016
Prediction loss at step 2892 is: 1.2922427654266357
Prediction loss at step 2893 is: 1.9634031057357788
Prediction loss at step 2894 is: 2.1344902515411377
Prediction loss at step 2895 is: 1.7042967081069946
Prediction loss at step 2896 is: 1.8680201768875122
Prediction loss at step 2897 is: 2.005751848220825
Prediction loss at step 2898 is: 1.9263248443603516
Prediction loss at step 2899 is: 1.930100440979004
Prediction loss a

In [11]:

# # Drift + means prediction is always more extreme;  drift - prediction is mild (in relative percentage)
# with tf.name_scope('metrics'):
#     drift = (preds[:,:,:3]-Y[:,:,:3])/Y[:,:,:3]
#     drift_truth_table = tf.to_float(tf.abs(drift) < drift_thredshold)
#     drift_within_t = tf.reduce_prod(drift_truth_table, axis=2)
#     correct_pred_ratio = tf.reduce_mean(drift_within_t)
    
# tf.summary.histogram('drift', drift)
# tf.summary.histogram('drift_within_threshold', drift_within_t)   
# tf.summary.scalar('loss',loss)
# tf.summary.scalar('correct_pred_ratio', correct_pred_ratio)
# merged = tf.summary.merge_all()

# # my_metrics = my_metrics(Y, preds)
# # p_x = my_metrics[0]
